In [21]:
import itertools
import math
import os
import gsd.hoomd
import hoomd
import numpy 
import warnings
import fresnel
import IPython
import packaging.version 
import matplotlib
import io 
import PIL
import freud
import random
import matplotlib.pyplot as plt 

%matplotlib inline 
matplotlib.style.use('ggplot')
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
import gsd.hoomd
import numpy as np
import os

position_A = [[0.25,0,0],
              [0.25,0.25,0],
              [0.25,0.5,0.25],
              [0,0.25,0.25],
              [0.25,0.25,0.5],
              [0,0,0.25]]
position_B = [[0.25,0,0],
              [0.25,0.25,0],
              [0.25,0.5,0.25],
              [0,0.25,0.25],
              [0.25,0.25,0.5],
              [0,0,0.25]]

positions = np.array(position_A + position_B, dtype=np.float32)

typeids = [0] * len(position_A) + [1] * len(position_B)

orientation = [(1, 0, 0, 0)] * len(positions)

frame = gsd.hoomd.Frame()

frame.particles.N = len(positions)  
frame.particles.position = positions
frame.particles.orientation = np.array(orientation, dtype=np.float32)
frame.particles.typeid = typeids  
frame.configuration.box = [10, 10, 10, 0, 0, 0]  
frame.particles.types = ['A', 'B']  

file_path = os.path.join(os.getcwd(), 'squareA&B.gsd')
if os.path.exists(file_path):
    os.remove(file_path)  

with gsd.hoomd.open(name=file_path, mode='x') as f:
    f.append(frame)

print("GSD file created with two shapes (A and B).")

GSD file 'two_shapes.gsd' created with two shapes (A and B).


In [23]:
cpu = hoomd.device.CPU()

In [24]:
simulation = hoomd.Simulation(device=cpu, seed=17)

In [25]:
simulation.create_state_from_gsd(filename='squareA&B.gsd')

fn = os.path.join(os.getcwd(), 'squareA&B_traj.gsd')
![ -e "$fn" ] && rm "$fn"

gsd_writer = hoomd.write.GSD(
    filename='squareA&B_traj.gsd', trigger=hoomd.trigger.Periodic(100), mode='xb'
)
simulation.operations.writers.append(gsd_writer)

In [26]:
integrator = hoomd.hpmc.integrate.ConvexSpheropolyhedronUnion(default_d=0.1, default_a=0.1)

vertices_1 = [(0.25, 0, 0), (0.25, 0.25, 0), (0.25, 0.5, 0.25),
              (0, 0.25, 0.25), (0.25, 0.25, 0.5), (0, 0, 0.25)]
vertices_2 = [(0.25, 0, 0), (0.25, 0.25, 0), (0.25, 0.5, 0.25),
              (0, 0.25, 0.25), (0.25, 0.25, 0.5), (0, 0, 0.25)]

positions = [(0, 0, 0), (0, 0, 0.1)]
orientations = [(1, 0, 0, 0), (1, 0, 0, 0)]
overlap = [1, 1]  

integrator.shape["A", "B"] = {
    "shapes": [{"vertices": vertices_1}, {"vertices": vertices_2}],
    "positions": positions,
    "orientations": orientations,
    "overlap": overlap
}

simulation.operations.integrator = integrator

simulation.run(1e4)



In [27]:
gsd_writer.flush()

In [28]:
simulation.timestep

10000